In [1]:
from datetime import datetime
import fsspec
import os
import re
import xarray as xr

from ormhw.core import DATA_DIR

In [2]:
grid_dir = os.path.join(DATA_DIR,'nasa/L2_GRIDDED')

In [3]:
years = range(2015,2024)
local = fsspec.filesystem('file')
for year in years:
    savefp = f"/home/jovyan/data/nasa/oc_{year}.nc"
    if os.path.isfile(savefp):
        continue
    gridded_files = local.glob(grid_dir + f'/oc_{year}*.nc')
    ds_list = []
    for filepath in gridded_files:
        day = re.findall(r'(\d{4}-\d{2}-\d{2})',filepath)[0]
        dt = datetime.strptime(day,'%Y-%m-%d')
        ds = xr.open_dataset(filepath)
        ds = ds.assign_coords({'time':[dt]})
        ds_list.append(ds)
    mds = xr.combine_by_coords(ds_list)
    mds = mds.sel(time = slice(datetime(year,7,9),datetime(year,10,8)))
    mds.to_netcdf(savefp)

In [4]:
ds_list = []
for year in years:
    _ds = xr.open_dataset(f"/home/jovyan/data/nasa/oc_{year}.nc")
    ds_list.append(_ds) 
mds = xr.combine_by_coords(ds_list)

In [6]:
mds = mds.sel(latitude = slice(39,51),longitude = slice(-128,-123.5))

In [8]:
mds.to_netcdf(f"/home/jovyan/data/nasa/oc_2015-2023.nc")